This google colab script was copied from Roboflow and adapted to my own storage structure in google drive.  


# Introduction
In this notebook, we implement [The TensorFlow 2 Object Detection Library](https://blog.tensorflow.org/2020/07/tensorflow-2-meets-object-detection-api.html) for training on your own dataset.

We also recommend reading our blog post on [Train TensorFlow 2 Object Detection on custom data](https://blog.roboflow.ai/training-yolov4-on-a-custom-dataset/) side by side.

We will take the following steps to implement YOLOv4 on our custom data:
* Install TensorFlow2 Object Detection Dependencies
* Download Custom TensorFlow2 Object Detection Dataset
* Write Custom TensorFlow2 Object Detection Training Configuation
* Train Custom TensorFlow2 Object Detection Model
* Export Custom TensorFlow2 Object Detection Weights
* Use Trained TensorFlow2 Object Detection For Inference on Test Images

When you are done you will have a custom detector that you can use. It will make inference like this:



#Mount google drive

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    base_path = "/content/gdrive/My Drive"
except:
    base_path = "C:/Users/Schiller/Google Drive/"
%cd {base_path}

# Edit configurations
Select, if a custom model should be used as pretrained model

In [ ]:
import ipywidgets as widgets
pretrained_db = widgets.Dropdown(
    options=[('Yes', 1), ('No', 2)],
    value=2,
    description='Use custom model as pretrained model:',
    style={'description_width': 'initial'}
)
display(pretrained_db)

In [ ]:
input_txt = "f"
while not input_txt == "":
    input_txt = input("Press enter to continue.")

Press enter to continue.


## Select configurations
If a custom created model used to continue training, ask for the name and load saved configurations, else run the configuration widgets.

In [ ]:
if pretrained_db.value == 1:
    # get possible trained custom models, quit if none was found
    import os
    from pathlib import Path
    pretrained_config_path = base_path + '/trained_models/'
    
    possible_models = []
    for dir_files in os.listdir( pretrained_config_path ):
        if dir_files.count(".") == 0:
            possible_models.append(dir_files)
    
    if len(possible_models) <= 0:
        print("No model found. Can not use a custom pretrained model.")
        quit()
    
    pretrained_model_select = widgets.Dropdown(
    options=possible_models,
    description='Pretrained model name:',
    style={'description_width': 'initial'}
    )
    display(pretrained_model_select)
    
else:
    # select model name 
    possible_models = [
                       'efficientdet-d0', 
                       'efficientdet-d1', 
                       'efficientdet-d2', 
                       'efficientdet-d3',
                       
                       'ssd_mobilenet_v2',
                       ]
    pretrained_model_select = widgets.Dropdown(
    options=possible_models,
    value='efficientdet-d0',
    description='Pretrained model name:',
    style={'description_width': 'initial'}
    )
    display(pretrained_model_select)
    
    # select batch size
    batch_size_slider = widgets.IntSlider(
        value=16, min=1, max=64, step=1,
        description='Batch-Size:',
        continuous_update=False,
        readout=True, readout_format='d'
    )
    display(batch_size_slider)
    
    # select dataset 
    possible_datasets = ['dataset-v1-dih4cps', 'dataset-v1-dih4cps']
    dataset_name_select = widgets.Dropdown(
        options=possible_datasets,
        value='dataset-v1-dih4cps',
        description='Dataset name:',
        style={'description_width': 'initial'}
        )
    display(dataset_name_select)

    # select num_eval_steps
    num_eval_steps_slider = widgets.IntSlider(
        value=500, min=10, max=1000, step=10,
        description='Number of steps before evaluation:',
        continuous_update=False,
        readout=True, readout_format='d',
        layout={'width': 'initial'},
        style={'description_width': 'initial'}
    )
    display(num_eval_steps_slider)

# select num_steps
num_steps_slider = widgets.IntSlider(
    value=1000, min=100, max=10000, step=100,
    description='Number of steps:',
    continuous_update=False,
    readout=True, readout_format='d',
    layout={'width': 'initial'},
    style={'description_width': 'initial'}
)
display(num_steps_slider)

Dropdown(description='Pretrained model name:', options=('efficientdet-d0', 'efficientdet-d1', 'efficientdet-d2…

IntSlider(value=16, continuous_update=False, description='Batch-Size:', max=64, min=1)

Dropdown(description='Dataset name:', options=('dataset-v1-dih4cps', 'dataset-v1-dih4cps'), style=DescriptionS…

IntSlider(value=500, continuous_update=False, description='Number of steps before evaluation:', layout=Layout(…

IntSlider(value=1000, continuous_update=False, description='Number of steps:', layout=Layout(width='initial'),…

In [ ]:
input_txt = "f"
while not input_txt == "":
    input_txt = input("Press enter to continue.")

Press enter to continue.


## Readout or load the configurations

In [ ]:
if pretrained_db.value == 1:
    #load config file
    pretrained_model_name = pretrained_model_select.value
    pretrained_config_path = base_path + '/trained_models/{}'.format(pretrained_model_name)
    %cd {pretrained_config_path}

    import json
    with open('model_config.txt') as json_file:
        cfg = json.load(json_file)
        with p in cfg['model_config']:
            chosen_model    = p['chosen_model']
            batch_size      = int(p['batch_size'])
            dataset_name    = p['dataset_name']
            num_done_steps  = int(p['num_done_steps'])
            num_eval_steps  = int(p['num_eval_steps'])
    num_steps = num_steps_slider.value  - num_done_steps
    if num_steps <= 0:
        print("{} steps allready done. Increase the number of steps.".format(num_done_steps))
        quit()
    
else:
    chosen_model = pretrained_model_select.value
    batch_size = batch_size_slider.value 
    dataset_name = dataset_name_select.value
    num_steps = num_steps_slider.value 
    num_eval_steps = num_eval_steps_slider.value 

import datetime
timestamp = datetime.datetime.now()
timestamp_str = "_{}-{}-{}".format(timestamp.year, 
                                    timestamp.month, 
                                    timestamp.day)
output_model_name = chosen_model + timestamp_str

print("Selected Model: {}".format(chosen_model))
print("Selected Dataset: {}".format(dataset_name))
print("Batch size: {}".format(batch_size))
print("Number of steps: {}".format(num_steps))
print("Number of steps before evaluation: {}".format(num_eval_steps))
print("Output model name/dir: {}".format(output_model_name))

Selected Model: ssd_mobilenet_v2
Selected Dataset: dataset-v1-dih4cps
Batch size: 16
Number of steps: 10000
Number of steps before evaluation: 500
Output model name/dir: ssd_mobilenet_v2_2020-11-13


# Install TensorFlow2 Object Detection Dependencies

In [ ]:
import os
import pathlib
%cd {base_path}
%mkdir 'tf2_object_detection_API'
%cd 'tf2_object_detection_API'

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models
!ls models

/content/gdrive/My Drive
mkdir: cannot create directory ‘tf2_object_detection_API’: File exists
/content/gdrive/My Drive/tf2_object_detection_API
AUTHORS     community	     ISSUES.md	official  README.md
CODEOWNERS  CONTRIBUTING.md  LICENSE	orbit	  research


In [ ]:
# Install the Object Detection API
%%bash
cd 'models/research/'
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/gdrive/My Drive/tf2_object_detection_API/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1598322 sha256=0d598b98ecb660c7a5ebe7de4c1dbf0d58e343c210d8e8535c02b69393aa13bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-wc9g69g8/wheels/80/09/2f/fd5551cbd065eb9f7ecd58a50ee37b2b745dbcbd80a6cac594
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import colab_utils

%matplotlib inline

In [ ]:
#run model builder test
%cd {base_path}
!python tf2_object_detection_API/models/research/object_detection/builders/model_builder_tf2_test.py


/content/gdrive/My Drive
2020-11-13 13:33:37.922522: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-11-13 13:33:40.828337: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-13 13:33:40.840155: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-11-13 13:33:40.840229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (be615d6c7d81): /proc/driver/nvidia/version does not exist
2020-11-13 13:33:40.848721: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2020-11-13 13:33:40.849000: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x16

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

#Select and prepare dataset
Define function to download the chosen dataset 

In [ ]:
from getpass import getpass
ACCESS_KEY      = "minio"
CLOUD_URL       = "https://minio.dih4cps.swms-cloud.com:9000/"
SECRET_KEY      = getpass("Enter the secret key for {}: ".format(ACCESS_KEY))

!pip install boto3
import boto3

def download_dataset(dataset_name, dataset_version):

    import os
    dataset_path = base_path + '/datasets'
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)
    dataset_path = os.path.join(dataset_path, dataset_name)
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)
    dataset_path = os.path.join(dataset_path, dataset_version)
    
    if os.path.exists(dataset_path):
        # dataset already downloaded
        return dataset_path
    
    ## if dataset not downloaded yet, download the selected dataset version
    os.mkdir(dataset_path)

    files_to_download = ['{}/train.record'.format(dataset_version), 
                         '{}/test.record'.format(dataset_version), 
                         '{}/images.txt'.format(dataset_version),
                         'labelmap.pbtxt']

    s3_client = boto3.client('s3',
            aws_access_key_id = ACCESS_KEY,
            aws_secret_access_key = SECRET_KEY,
            endpoint_url=CLOUD_URL, 
            verify=True,
            config=boto3.session.Config(signature_version='s3v4'))

    for f in files_to_download:
        s3_client.download_file(dataset_name,           # bucket name
                f,                                      # file key from cloud
                dataset_path + '/' + f.split("/")[-1])  # local filename
                                
        
    return base_path + '/datasets/' + dataset_name

def get_possible_dataset_versions(dataset_name):
    versions = []
    s3_resource = boto3.resource('s3',
            aws_access_key_id = ACCESS_KEY,
            aws_secret_access_key = SECRET_KEY,
            endpoint_url=CLOUD_URL, 
            verify=True,
            config=boto3.session.Config(signature_version='s3v4'))
    bucket = s3_resource.Bucket(dataset_name)

    for bucket_object in bucket.objects.all():
            object_name = str(bucket_object.key)
            version = object_name.split("/")[0]
            if versions.count(version) <= 0 and version.count("version") > 0:
                versions.append(version)

    return versions 
        

##Select a dataset version  

In [ ]:
possible_versions = get_possible_dataset_versions(dataset_name)
if len(possible_versions) <= 0:
    print("No dataset version available, check the dataset in the minio cloud.")
 
dataset_version_select = widgets.Dropdown(
        options=possible_versions,
        description='Dataset version:',
        style={'description_width': 'initial'}
        )
display(dataset_version_select)

Dropdown(description='Dataset version:', options=('version_2020-10-15', 'version_2020-10-27', 'version_2020-10…

In [ ]:
input_txt = "f"
while not input_txt == "":
    input_txt = input("Press enter to continue.")

Press enter to continue.


## Download the dataset and set paths 

In [ ]:
dataset_version = dataset_version_select.value
dataset_path = download_dataset(dataset_name, dataset_version)
test_record_fname = dataset_path + '/test.record'
train_record_fname = dataset_path + '/train.record'
label_map_pbtxt_fname = dataset_path + '/labelmap.pbtxt'
print(dataset_path)

/content/gdrive/My Drive/datasets/dataset-v1-dih4cps/version_2020-11-09


# Configure Custom TensorFlow2 Object Detection Training Configuration




> In this section you can specify any model in the [TF2 OD model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and set up your training configuration.



In [ ]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz', 
        'batch_size': 16
    },
} # if you add a new model, also add the model short name to selection dropdown 


#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!

pretrained_model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
#download pretrained weights
%cd {base_path}

pretrained_path = "./tf2_object_detection_API/pretrained_models/"
%mkdir {pretrained_path}
%cd {pretrained_path}

import tarfile
import os

if not os.path.exists(pretrained_checkpoint):
    download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
    !wget {download_tar}
else:
    print("Already downloaded {}".format(pretrained_checkpoint))

if not os.path.exists(pretrained_checkpoint.split(".")[0]):
    tar = tarfile.open(pretrained_checkpoint)
    tar.extractall()
    tar.close()
else:
    print("Already extracted {}".format(pretrained_checkpoint.split(".")[0]))
    

/content/gdrive/My Drive
mkdir: cannot create directory ‘./tf2_object_detection_API/pretrained_models/’: File exists
/content/gdrive/My Drive/tf2_object_detection_API/pretrained_models
Already downloaded ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Already extracted ssd_mobilenet_v2_320x320_coco17_tpu-8


In [ ]:
#download base training configuration file
import os
if not os.path.exists(base_pipeline_file):
    download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
    !wget {download_config}
else:
    print("Already downloaded {}".format(base_pipeline_file))

Already downloaded ssd_mobilenet_v2_320x320_coco17_tpu-8.config


In [ ]:
#prepare
pipeline_fname = pretrained_path + base_pipeline_file
fine_tune_checkpoint = pretrained_path + pretrained_model_name + '/checkpoint/ckpt-0'
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd {base_path}
%mkdir created_models
%cd created_models
%mkdir {output_model_name}
%cd {base_path}
print('writing custom configuration file')
print(pipeline_fname)
with open(pipeline_fname) as f:
    s = f.read()
created_config_path = 'created_models/{}/pipeline_file.config'.format(output_model_name)
with open(created_config_path, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)



/content/gdrive/My Drive
mkdir: cannot create directory ‘created_models’: File exists
/content/gdrive/My Drive/created_models
/content/gdrive/My Drive
writing custom configuration file
./tf2_object_detection_API/pretrained_models/ssd_mobilenet_v2_320x320_coco17_tpu-8.config


In [ ]:
%cd {base_path}
file_path = './created_models/{}/pipeline_file.config'.format(output_model_name)
%cat {file_path}

/content/gdrive/My Drive
# SSD with Mobilenet v2
# Trained on COCO17, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
   

In [ ]:
%cd {base_path}
pipeline_file = './created_models/{}/pipeline_file.config'.format(output_model_name)
model_dir = './created_models/{}/training'.format(output_model_name)

/content/gdrive/My Drive


# Train Custom TF2 Object Detector

* pipeline_file: defined above in writing custom training configuration
* model_dir: the location tensorboard logs and saved model checkpoints will save to
* num_train_steps: how long to train for
* num_eval_steps: perform eval on validation set after this many steps







In [ ]:
%cd {base_path}
print("Training model with")
print("config:             {}".format(pipeline_file))
print("model directory:    {}".format(model_dir))
print("Number of steps:    {}".format(num_steps))
print("Number eval steps:  {}".format(num_eval_steps))
!python tf2_object_detection_API/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}


/content/gdrive/My Drive
Training model with
config:             ./created_models/ssd_mobilenet_v2_2020-11-13/pipeline_file.config
model directory:    ./created_models/ssd_mobilenet_v2_2020-11-13/training
Number of steps:    10000
Number eval steps:  500
2020-11-13 13:36:50.676415: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-13 13:36:53.303707: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-13 13:36:53.314383: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-11-13 13:36:53.314442: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (be615d6c7d81): /proc/driver/nvidia/version does not exist
2020-11-13 13:36:53.322835: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Fr

In [ ]:
#run model evaluation to obtain performance metrics
#!python /content/models/research/object_detection/model_main_tf2.py \
    #--pipeline_config_path={pipeline_file} \
    #--model_dir={model_dir} \
    #--checkpoint_dir={model_dir} \
#Not yet implemented for EfficientDet

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir model_dir+'/train'

## Export configurations

In [ ]:
# make output dir 
import os
output_directory = './trained_models/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
output_directory += output_model_name
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
%cd {output_directory}

import json
"""
    ToDo: update done_num_steps
"""
cfg = {}
cfg['model_config'] =  []
cfg['model_config'].append({
    'base_model'      : chosen_model, 
    'batch_size'      : batch_size,
    'num_steps'       : num_steps,
    'done_num_steps'  : num_steps,
    'num_eval_steps'  : num_eval_steps,
    'output_model'    : output_model_name})

with open('model_config.txt', 'w') as outfile:
    json.dump(cfg, outfile)

## Exporting a Trained Inference Graph
Still to come for TF2 models, we will be updating this Colab notebook accordingly as the functionality is added. 

In [ ]:
#see where our model saved weights
path = base_path + '/created_models/training/'
%ls {path}

In [ ]:
#run conversion script
import re
import numpy as np

%cd {base_path}
output_directory = './trained_models'

#place the model weights you would like to export here
last_model_path = './created_models/training/'
print(last_model_path)
!python tf2_object_detection_API/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [ ]:
%ls './trained_models/saved_model/'


## Export trained model to tflite 

In [ ]:
#recover our saved model
pipeline_config = base_path + '/trained_models/pipeline.config'
#generally you want to put the last ckpt from training in here
model_dir = base_path + '/trained_models/saved_model'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# get latest ckpt 
import glob 
ckpt_path = base_path + '/trained_models/checkpoint/'
ckpts = sorted(glob.glob(ckpt_path + 'ckpt-*.index'))
latest_ckpt = ckpts[-1].split(".")[0]
print(latest_ckpt)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(latest_ckpt)

class Squared(tf.Module):
  @tf.function
  def __call__(self, image):
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])
model = Squared()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
concrete_func = model.__call__.get_concrete_function()

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()


In [ ]:
import tensorflow as tf

# Create a model using low-level tf.* APIs
class Squared(tf.keras.Model):
  @tf.function
  def call(self, inputs):
    return tf.square(inputs)
model = Squared()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
concrete_func = model.call.get_concrete_function()

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()


In [ ]:
!pip install tensorflow
import tensorflow as tf
saved_model_dir = base_path + "/trained_models/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir, signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
%%bash
cd '/content/gdrive/My Drive/tf2_object_detection_API/models/research/object_detection/'

python export_tflite_ssd_graph.py \
--pipeline_config_path='/content/gdrive/My Drive/trained_models/pipeline.config' \
--trained_checkpoint_prefix='/content/gdrive/My Drive/trained_models/checkpoint/model.ckpt-0' \
--output_directory='/content/gdrive/My Drive/trained_models/tflite' \
--add_postprocessing_op=true

# Run Inference on Test Images with Custom TensorFlow2 Object Detector

In [ ]:
path = base_path + '/test/'
%mkdir {path}
%cd {path}

In [ ]:

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
#recover our saved model
pipeline_config = base_path + '/trained_models/pipeline.config'
#generally you want to put the last ckpt from training in here
model_dir = base_path + '/trained_models/saved_model'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# get latest ckpt 
import glob 
ckpt_path = base_path + '/trained_models/checkpoint/'
ckpts = sorted(glob.glob(ckpt_path + 'ckpt-*.index'))
latest_ckpt = ckpts[-1].split(".")[0]
print(latest_ckpt)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(latest_ckpt)


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random

TEST_IMAGE_PATHS = glob.glob(base_path + '/test/*.png')
image_path = random.choice(TEST_IMAGE_PATHS)
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

[Roboflow](https://roboflow.ai)
